In [18]:
SHARED="/data/sharing/QuakeCoRE"

import sys

sys.path.append(f"{SHARED}/Ancillary_tools/CPT_Vsz_Vs30")
sys.path.append(f"{SHARED}/Ancillary_tools")
sys.path.append(f"{SHARED}/qcore")

from collections import Counter
from time import strftime,localtime
import os
import os.path
from pathlib import Path
import yaml


from matplotlib import pyplot as plt
import numpy as np
import pandas as pd


from sqlalchemy import Column, ForeignKey, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine, desc
from sqlalchemy.orm import sessionmaker


from getCPTdata import getCPTdata
from computeVs import Vs_McGann
from computeVsz import compute_vsz_from_vs
from computeVs30 import vsz_to_vs30
from loc_filter import locs_multiple_records
from qcore import geo


HOME = Path(os.path.expanduser("~"))
out_dir = HOME / "Data/cpt/outdir"
plot_dir = out_dir / "validation_plots"

plot_dir.mkdir(parents=True, exist_ok=True)



results = {}



engine = create_engine(f'sqlite:///{SHARED}/Data/nz_cpt.db')
DBSession = sessionmaker(bind=engine)
session = DBSession()
                


def log_error(skipped_fp, cpt_name, error):
    skipped_fp.write(f"{cpt_name} - {error}\n")


def count_digits(arr):
    stringified = str(arr).replace("0", "").replace(".", "")
    return Counter(stringified)


Base = declarative_base()

class CPTLocation(Base):
    __tablename__ = 'cpt_location'
    id = Column(Integer, primary_key=True)
    #    customer_id=Column(Integer, ForeignKey('customers.id'))
    name = Column(String(20), nullable=False)  # 20210427_17
    private = Column(Integer) #true / false
    type = Column(String(5)) # CPT or SCPT
    nztm_x = Column(Float)
    nztm_y = Column(Float)

    def __iter__(self): #overridding this to return tuples of (key,value)
        return iter([('id',self.id),('name',self.name),('nztm_x',self.nztm_x),('nztm_y',self.nztm_y)])


class CPTDepthRecord(Base):
    __tablename__ = 'cpt_depth_record'
    id = Column(Integer, primary_key=True)
    cpt_name = Column(String(20), nullable=False)  # 
    depth = Column(Float) #
    qc = Column(Float) #
    fs = Column(Float)
    u = Column(Float)
    loc_id = Column(Integer, ForeignKey('cpt_location.id'))
    
    def __iter__(self): #overridding this to return tuples of (key,value)
        return iter([('id',self.id),('depth',self.depth),('qc',self.qc),('fs',self.fs),('u',self.u),('loc_id',self.loc_id)])


# not really useful, but presented as an example
def cpt_records(cpt_name):
    res=session.query(CPTDepthRecord).filter(CPTDepthRecord.cpt_name == cpt_name).all()
    return res

# not really useful, but presented as an example
def max_depth_record(cpt_name):
    res=session.query(CPTDepthRecord).filter(CPTDepthRecord.cpt_name == cpt_name).order_by(CPTDepthRecord.depth.desc()).first()
    return res

# the following 3 functions are actually used
def cpt_locations():
    return session.query(CPTLocation).all()

def cpt_records_exists(cpt_name):
    res=session.query(CPTDepthRecord).filter(CPTDepthRecord.cpt_name == cpt_name).first()
    return (res is not None)

def get_cpt_data(cpt_name, columnwise=True):
    res=session.query(CPTDepthRecord.depth,CPTDepthRecord.qc,CPTDepthRecord.fs,CPTDepthRecord.u).filter(CPTDepthRecord.cpt_name == cpt_name).all()
    res_array =np.array(res)
    if columnwise: #each column is grouped together
        res_array = res_array.T
    return res_array
    


# Usage Examples

Check if any record found for a CPT name

In [19]:
print(cpt_records_exists("CPT_108566"))
print(cpt_records_exists("SCPT_140251"))


False
True


In [20]:
get_cpt_data("CPT_108566")


array([], dtype=float64)

Retrieve CPT records for a given CPT

In [21]:
%%time
res=cpt_records('SCPT_140251')
#print(res)
#print(res[0].depth)

CPU times: user 7.98 ms, sys: 8.01 ms, total: 16 ms
Wall time: 15.7 ms


However, the following function is preferred

In [22]:
depth, qc, fs, u = get_cpt_data("SCPT_140251")

In [23]:
print(depth)

[ 0.03  0.05  0.07 ... 20.65 20.67 20.69]


With `columnwise=False`, each CPT depth record is grouped together as below. `columnwise=True` by default

In [24]:
cpt_records = get_cpt_data("SCPT_140251", columnwise=False)
print(cpt_records[0])

[ 0.03    0.0105  0.0002 -0.0004]


Retrieves all locations

In [25]:
%%time
locs = cpt_locations()
print(dict(locs[0]))


{'id': 1, 'name': 'CPT_1', 'nztm_x': 1576467.294706431, 'nztm_y': 5181262.382226084}
CPU times: user 1.36 s, sys: 36 ms, total: 1.4 s
Wall time: 1.4 s


The record at the maximum depth can be retrived by this function. (In practice, this function may not be useful)

In [26]:
%%time
print(dict(max_depth_record('SCPT_140251')))


{'id': 1034, 'depth': 20.69, 'qc': 22.7256, 'fs': 1.0416, 'u': -0.0251, 'loc_id': 39741}
CPU times: user 4.59 ms, sys: 11 µs, total: 4.6 ms
Wall time: 3.51 ms


# Find Duplicate Locations

In [27]:
#dup_locs_dict is a dictionary {loc0:[loc00,loc01],loc1:[loc10,loc11,loc12],loc2:[loc20]...} 
#meaning loc00,loc01 are within 0.1m distance from loc0. It may be too strict.

#computing locs_multiple_records() takes about 5 mins, and I'm skipping here
#if filtering criteria needs to be updated, revise locs_multiple_records() @ loc_filter.py and delete out_dir/dup_locs.yaml

dup_locs_yaml_file = out_dir/"dup_locs.yaml"

if dup_locs_yaml_file.exists():
    with open(out_dir/"dup_locs.yaml", 'r') as f:
        dup_locs_dict = yaml.load(f, Loader=yaml.SafeLoader)
else:        
    dup_locs_dict=locs_multiple_records(locs, stdout=True)
    #let's save this in a yaml file for future use
    with open(out_dir/"dup_locs.yaml","w") as f:
        yaml.safe_dump(dup_locs_dict,f)

    
#flattens dictionary into lists. [loc00,loc01,loc10,loc11,loc12,loc20...]
import functools,operator
dup_locs = functools.reduce(operator.iconcat, list(dup_locs_dict.values()), []) 

#note that loc0,loc1 are not in dup_locs. These locations are to be processed. Duplicates are to be skipped


# Main routine

Loops through all `locs`, and for each location, it performs filtering process. When all the filtering criteria is met, it does conversion from CPT to Vs, Vsz and Vs30

In [29]:
%%time

timestamp=strftime("%Y%m%d_%H%M", localtime())

skipped_fp = open(out_dir / f"skipped_cpts_{timestamp}", "w")
output_file = out_dir / f"vs30_results_{timestamp}.csv"

log_error(skipped_fp, "<<<<<", f"Beginning loop : {timestamp}")
for row_n, loc in enumerate(locs):
    
    cpt_name = loc.name
    if not cpt_records_exists(cpt_name):
        log_error(skipped_fp, cpt_name, f"Type 01 : No record found: {cpt_name}")
        continue
    
    if row_n % 1000 == 0: #print every 1000
        print(f"{row_n+1}/{len(locs)} - {cpt_name}")
    
    z, qc, fs, u2 = get_cpt_data(cpt_name)
     

    # duplicate location
    if cpt_name in dup_locs:
        log_error(skipped_fp, cpt_name, f"Type 02 :Duplicate location")
        continue

    # duplicate depth check
    u, c = np.unique(z, return_counts=True)
    if np.any([c > 1]):
        log_error(skipped_fp, cpt_name, f"Type 03 : Duplicate depth detected - invalid CPT")
        continue

    # Check for invalid negative readings
    if any(fs < -0.2) or any(qc < -0.2) or any(u2 < -0.2):
        log_error(skipped_fp, cpt_name, f"Type 04 : negative value - discarding")
        continue

    # Check for repeated digits
    if any(value > 3 for fs_value in fs for value in count_digits(fs_value).values()):
        log_error(skipped_fp, cpt_name, f"Type 05 : Repeated digit - investigating")
        continue

    max_depth = max(z)
    if max_depth < 5:
        log_error(skipped_fp, cpt_name, f"Type 06 : depth<5: {max_depth}")
        continue
    min_depth = min(z)
    z_span = max_depth - min_depth
    if z_span < 5:
        log_error(skipped_fp, cpt_name, f"Type 07 : depth range <5: {z_span}")
        continue
        
    #All the filtering is complete and this record has survived
    
    (z, Vs, Vs_SD) = Vs_McGann(z, qc, fs)
                                                   
    Vsz, max_depth = compute_vsz_from_vs(Vs, z)
    vs30 = vsz_to_vs30(Vsz, z)

    vs30_result = {}
    vs30_result["NZTM_X"] = loc.nztm_x
    vs30_result["NZTM_Y"] = loc.nztm_y
    vs30_result["Vsz"] = Vsz
    vs30_result["Vs30"] = vs30
    vs30_result["Zmax"] = max_depth
    vs30_result["Zmin"] = min_depth
    vs30_result["Zspan"] = z_span
    results[cpt_name] = vs30_result
    if row_n < 10:
        fig, ax = plt.subplots()
        ax.plot(fs, z)
        ax.invert_yaxis()
        ax.set_ylabel("Depth")
        ax.set_xlabel("fs")
        ax.grid()
        fig.savefig(plot_dir / f"{cpt_name}_fs.png")
        plt.close(fig)

        fig, ax = plt.subplots()
        lowerVs = np.exp(np.log(Vs) - Vs_SD)
        upperVs = np.exp(np.log(Vs) + Vs_SD)
        ax.plot(Vs, z, "red")
        ax.plot(lowerVs, z, "r--", linewidth=0.5)
        ax.plot(upperVs, z, "r--", linewidth=0.5)
        ax.grid()
        ax.invert_yaxis()
        ax.set_ylabel("Depth")
        ax.set_xlabel("Vs (m/s)")
        ax.set_xlim(0, 600)
        fig.savefig(plot_dir / f"{cpt_name}_Vs.png")
        plt.close(fig)

log_error(skipped_fp, ">>>>>", f"Ending loop : {timestamp}")

1/49321 - CPT_1
1001/49321 - CPT_103928
3001/49321 - CPT_11089


KeyboardInterrupt: 

# Save Vs30 Estimates

In [30]:
result_df = pd.DataFrame.from_dict(results, orient="index")
result_df.to_csv(output_file)

ll = geo.wgs_nztm2000x(result_df[["NZTM_X", "NZTM_Y"]])
result_df["lon"] = ll[:, 0]
result_df["lat"] = ll[:, 1]

result_df.plot.scatter("Vs30", "Vsz")
fig = plt.gcf()
fig.savefig(plot_dir / "vs30_vsz_scatter.png")

KeyError: "None of [Index(['NZTM_X', 'NZTM_Y'], dtype='object')] are in the [columns]"

In [ ]:
result_df